#setup

In [ ]:

from google.colab import drive
drive.mount('/content/drive')
!pip install transformers

# Imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd


from scipy.sparse import hstack
from scipy.sparse import csr_matrix


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
# Filter warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score



In [ ]:
!pip install optuna

In [ ]:
import optuna

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import BertTokenizerFast, BertModel, AdamW
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from collections import Counter


In [ ]:
torch.manual_seed(42)
np.random.seed(42)

# specify GPU
device = torch.device("cuda")

In [ ]:
# Reading data
folder_path = '/content/drive/MyDrive/datasets/'
dfTrain = pd.read_csv(folder_path + 'trainR.csv')
dfTest = pd.read_csv(folder_path + 'testR.csv')

dfTrain.drop(['ID_LAT_LON_YEAR_WEEK'],axis=1,inplace=True)

dfTest.drop(['ID_LAT_LON_YEAR_WEEK'],axis=1,inplace=True)

In [ ]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 75 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   latitude                                                  79023 non-null  float64
 1   longitude                                                 79023 non-null  float64
 2   year                                                      79023 non-null  int64  
 3   week_no                                                   79023 non-null  int64  
 4   SulphurDioxide_SO2_column_number_density                  64414 non-null  float64
 5   SulphurDioxide_SO2_column_number_density_amf              64414 non-null  float64
 6   SulphurDioxide_SO2_slant_column_number_density            64414 non-null  float64
 7   SulphurDioxide_cloud_fraction                             64414 non-null  float64
 8   SulphurDioxide_s

In [ ]:
dfTest

,latitude,longitude,year,week_no,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
0,-0.510,29.290,2022,0,NaN,NaN,NaN,NaN,NaN,NaN,...,36022.027344,8472.313477,41047.937500,7472.313477,7.935617,0.240773,-100.113792,33.697044,-133.047546,33.779583
1,-0.510,29.290,2022,1,0.000456,0.691164,0.000316,0.000000,76.239196,15.600607,...,48539.737242,6476.147323,54915.708579,5476.147161,11.448437,0.293119,-30.510319,42.402593,-138.632822,31.012380
2,-0.510,29.290,2022,2,0.000161,0.605107,0.000106,0.079870,-42.055341,39.889060,...,34133.080469,8984.795703,39006.093750,7984.795703,10.753179,0.267130,39.087361,45.936480,-144.784988,26.743361
3,-0.510,29.290,2022,3,0.000350,0.696917,0.000243,0.201028,72.169566,58.862543,...,50854.991076,6014.724059,57646.368368,5014.724115,11.764556,0.304679,-24.465127,42.140419,-135.027891,29.604774
4,-0.510,29.290,2022,4,-0.000317,0.580527,-0.000184,0.204352,76.190865,15.646016,...,46594.685145,6849.280477,52896.541873,5849.280394,13.065317,0.284221,-12.907850,30.122641,-135.500119,26.276807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24348,-3.299,30.301,2022,44,-0.000618,0.745549,-0.000461,0.234492,72.306198,61.114494,...,48839.430415,6260.120033,55483.459980,5260.120056,30.398508,0.180046,-25.528588,45.284576,-116.521412,29.992562
24349,-3.299,30.301,2022,45,NaN,NaN,NaN,NaN,NaN,NaN,...,47042.694849,6678.843299,53589.917383,5678.951521,19.223844,0.177833,-13.380005,43.770351,-122.405759,29.017975
24350,-3.299,30.301,2022,46,NaN,NaN,NaN,NaN,NaN,NaN,...,55337.148173,5336.282475,62646.761340,4336.282491,13.801194,0.219471,-5.072065,33.226455,-124.530639,30.187472
24351,-3.299,30.301,2022,47,0.000071,1.003805,0.000077,0.205077,74.327427,38.215228,...,44813.691428,7188.578533,50728.313991,6188.578464,27.887489,0.247275,-0.668714,45.885617,-129.006797,30.427455


#funcers


In [ ]:

def get_first_non_nan_sorted(dataset):
    total_rows = len(dataset)

    # Initialize a list to store the first non-NaN values and their indices
    first_non_nan_values = []

    for column in dataset.columns:
        # Get the column values excluding NaNs
        column_values = dataset[column].values
        non_nan_indices = np.where(~np.isnan(column_values))[0]

        # Check if there are any non-NaN values
        if len(non_nan_indices) > 0:
            # Retrieve the first non-NaN value and its index
            first_non_nan_value = column_values[non_nan_indices[0]]
            index_type = type(dataset.index[0])
            first_non_nan_index = index_type(dataset.index[non_nan_indices[0]])

            # Add the first non-NaN value and its index to the list
            first_non_nan_values.append((column, first_non_nan_index, first_non_nan_value))

    # Sort the first non-NaN values based on the sorted indices
    sorted_first_non_nan_values = sorted(first_non_nan_values, key=lambda x: x[1])

    return sorted_first_non_nan_values



def plot_columns(dataframe, n=10, figsize=(20, 16)):
    # Apply min-max scaling to the selected columns
    scaler = MinMaxScaler()
    scaled_columns = pd.DataFrame(scaler.fit_transform(dataframe),
                                  columns=dataframe.columns,
                                  index=dataframe.index)

    # Number of graphs
    num_graphs = len(dataframe.columns) // n if len(dataframe.columns) % n == 0 else len(dataframe.columns) // n + 1


    for k in range(num_graphs):
        start = k * n
        end = (k + 1) * n if (k + 1) * n < len(dataframe.columns) else len(dataframe.columns)

        fig, ax = plt.subplots(figsize=figsize)

        # Plot each column as a line
        for column in dataframe.columns[start:end]:
            ax.plot(scaled_columns.index, scaled_columns[column], label=column)

        # Set the x-axis label
        ax.set_xlabel('Index')

        # Set the y-axis label
        ax.set_ylabel('Scaled Value')

        # Set the title
        ax.set_title(f'Columns Plot {k+1}')

        # Add a legend
        ax.legend()

        # Display the plot
        plt.show()
def print_nan_zero_percentage(dataset):
    total_rows = len(dataset)

    # Calculate percentage of NaN and 0 values for each column
    nan_zero_percentage = []
    for column in dataset.columns:
        nan_percentage = dataset[column].isna().sum() / total_rows * 100
        zero_percentage = (dataset[column] == 0).sum() / total_rows * 100

        nan_zero_percentage.append((column, nan_percentage, zero_percentage))

    # Sort the percentages in descending order based on NaN percentage
    sorted_nan_zero_percentage = sorted(nan_zero_percentage, key=lambda x: x[1], reverse=True)

    # Print the sorted percentages
    for column, nan_percentage, zero_percentage in sorted_nan_zero_percentage:
        print(f"Column: {column}")
        print(f"NaN Percentage: {nan_percentage:.2f}%")
        print(f"Zero Percentage: {zero_percentage:.2f}%")
        print("------------------------")

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

def plot_yearly(df, start, end, start_date, end_date):
    # Plot each column
    for column in df.columns[start:end+1]:
        # Create a new figure for this column
        fig, ax = plt.subplots(figsize=(12, 6))

        # For each year
        for year in range(pd.to_datetime(start_date).year, pd.to_datetime(end_date).year):
            # Get data for this year
            start_of_year = max(pd.to_datetime(f"{year}-01-01"), pd.to_datetime(start_date))
            end_of_year = min(pd.to_datetime(f"{year+1}-01-01") - pd.Timedelta(days=1), pd.to_datetime(end_date))
            yearly_data = df[(df.index >= start_of_year) & (df.index <= end_of_year)][column]

            # Apply min-max scaling to the selected columns
            scaler = MinMaxScaler()
            yearly_data = pd.Series(scaler.fit_transform(yearly_data.values.reshape(-1, 1)).flatten(),
                                    index=yearly_data.index)

            # Create a continuous date index and interpolate missing data
            all_days = pd.date_range(start=start_of_year, end=end_of_year, freq='D')
            yearly_data = yearly_data.reindex(all_days).interpolate(method='time')

            # Plot data
            ax.plot(yearly_data.index.dayofyear, yearly_data.values, label=year)

        # Set labels and title
        ax.set_xlabel('Day of Year')
        ax.set_ylabel('Scaled Value')
        ax.set_title(f'{column} - Yearly Comparison')

        # Simplify x-axis to 12 ticks
        ax.set_xticks(np.linspace(1, 365, num=12))
        ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

        # Show legend
        ax.legend()

        # Show the plot
        plt.show()

import matplotlib.patches as mpatches

def visualize_data(df, figsize=(15, 10)):
    n = 10  # Number of columns per plot
    num_plots = np.ceil(df.shape[1] / n).astype(int)  # Calculate the number of plots needed

    for i in range(num_plots):
        fig, ax = plt.subplots(figsize=figsize)  # Set the figure size

        # Select the columns for this plot
        columns = df.columns[i * n: (i + 1) * n]

        for col_num, col_name in enumerate(columns):
            col_data = df[col_name]

            # Iterate over rows and stack them in the bar plot
            for row_num, value in enumerate(col_data):
                if pd.isna(value):
                    color = 'red'
                elif value == 0:
                    color = 'blue'
                else:
                    color = 'green'
                ax.bar(col_num, 1, bottom=row_num, color=color)

        # Set Y-axis labels as row indices for selected rows only
        selected_ticks = range(0, len(df.index), len(df.index)//10)  # Select every 10% of data for labeling
        ax.set_yticks(selected_ticks)
        ax.set_yticklabels(df.index[selected_ticks])

        # Set X-axis labels as column names
        ax.set_xticks(range(len(columns)))
        ax.set_xticklabels(columns)

        # Create legend
        red_patch = mpatches.Patch(color='red', label='NaN')
        blue_patch = mpatches.Patch(color='blue', label='Zero')
        green_patch = mpatches.Patch(color='green', label='Non-Zero')
        ax.legend(handles=[red_patch, blue_patch, green_patch])

        plt.show()

In [ ]:
from google.colab import files

def create_and_download_submission(predictions, filename='submission.csv'):
    # Read the CSV file that contains the ID of each test sample
    dfid = pd.read_csv(folder_path + 'testR.csv')

    # Create a dataframe for the submission
    submission = pd.DataFrame({
        "ID_LAT_LON_YEAR_WEEK": dfid["ID_LAT_LON_YEAR_WEEK"],
        "emission": predictions
    })

    # Save the submission dataframe to a CSV file
    submission.to_csv(filename, index=False)

    # Download the CSV file
    files.download(filename)

In [ ]:
def print_column_summary(df):
    for col in df.columns:
        unique_values = df[col].nunique()
        top_values = df[col].value_counts(dropna=False).head(5).to_dict()
        top_values_str = "///".join([f'[{str(k)} = {v}]' for k, v in top_values.items()])

        print(f'Column: {col} - number of unique values = {unique_values}/ Top 5 {top_values_str}')


In [ ]:
import pandas as pd

def fill_nans_with_previous(df):
    for column in df.columns:
        # Fill NaNs with the previous non-NaN values
        df[column].fillna(method='ffill', inplace=True)

        # If the column starts with NaN values, fill with the next non-NaN value
        if pd.isna(df[column].iloc[0]):
            df[column].fillna(method='bfill', inplace=True)

    return df



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def eda_function(df, target_column,
                 figsize_heatmap=(12, 8),
                 figsize_corr=(20, 20),
                 figsize_hist=(20, 20),
                 bins_hist=20,
                 figsize_scatter=(20, 20),
                 selected_features=None,
                 num_hist_per_fig=9,
                 num_scatter_per_fig=6,
                 num_features_per_heatmap=10,
                 num_features_per_pairplot=6):


    print('*'*180)
    print_nan_zero_percentage(df)
    print('*'*180)
    # Summary statistics
    print(df.describe())

    print('*'*180)
    print('*'*180)
    print()

    # Histograms for numerical features
    numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
    for i in range(0, len(numerical_cols), num_hist_per_fig):
        df[numerical_cols[i:i + num_hist_per_fig]].hist(figsize=figsize_hist, bins=bins_hist)
        plt.show()
        print('-'*180)




    print('*'*180)
    print('*'*180)
    print()

    # Scatter plots to observe relationships
    for i in range(0, len(numerical_cols) - 1, num_scatter_per_fig):
        fig, axes = plt.subplots(1, num_scatter_per_fig, figsize=figsize_scatter, sharey=True)
        for j in range(num_scatter_per_fig):
            if i + j < len(numerical_cols) - 1:
                column = numerical_cols[i + j]
                axes[j].scatter(df[column], df[target_column])
                axes[j].set_xlabel(column)
                axes[j].set_ylabel(target_column)
                axes[j].set_title(f"{column} vs {target_column}")
        plt.show()
        print('-'*180)

    print('*'*180)
    print('*'*180)
    print()

    # Pairplot for selected features (avoiding all for heavy computational load)

    numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.drop(target_column)
    for i in range(0, len(numerical_columns), num_features_per_pairplot):
        selected_features = numerical_columns[i:i+num_features_per_pairplot].to_list()
        selected_features.append(target_column)
        sns.pairplot(df[selected_features], y_vars=target_column, x_vars=selected_features)
        plt.show()

def analyze_nans(df):
    results = {}
    for col in df.columns:
        # Count consecutive NaNs
        max_consecutive_nans = 0
        current_consecutive_nans = 0
        # Count total NaNs for average calculation
        total_nans = 0

        for value in df[col]:
            if pd.isna(value):
                current_consecutive_nans += 1
                total_nans += 1
            else:
                if current_consecutive_nans > max_consecutive_nans:
                    max_consecutive_nans = current_consecutive_nans
                current_consecutive_nans = 0

        if current_consecutive_nans > max_consecutive_nans:
            max_consecutive_nans = current_consecutive_nans

        # Calculate average number of NaNs per 365 elements
        average_nans_per_365 = total_nans / (len(df[col]) / 365) if len(df[col]) > 0 else 0

        results[col] = {
            'largest_number_of_nans_in_a_row': max_consecutive_nans,
            'average_number_of_nans_per_365_elements': average_nans_per_365
        }

    return results



In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

def plot_top_correlations(df, target_column, num_features_per_heatmap):
    # Drop columns with over 70% NaN values
    df = df.dropna(thresh=len(df) * 0.2, axis=1)

    # Drop remaining NaN values
    df = df.dropna()

    # Compute the correlation matrix
    corrmat = df.corr()

    # Get the correlations with the target column, sorted by absolute value
    sorted_correlations = corrmat[target_column].apply(abs).sort_values(ascending=False)
    for i in range(1, len(sorted_correlations), num_features_per_heatmap):
        # Get the next set of correlated features including the target column
        cols = [target_column] + sorted_correlations.index[i:i + num_features_per_heatmap].tolist()
        cm = np.corrcoef(df[cols].values.T)
        sns.set(font_scale=0.8) # Adjusting the font scale for smaller labels
        plt.figure(figsize=(20,20)) # Making the figure size larger
        hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.3f', annot_kws={'size': 8}, # Smaller annotation size
                        yticklabels=cols, xticklabels=cols, cmap='coolwarm')
        plt.title(f'Heatmap of Top {num_features_per_heatmap} Correlations with {target_column}')
        plt.show()
        print("\n" * 3 + "=" * 180 + "\n" * 3)








# 10 layers you crazey!



In [ ]:
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
# Loss function for regression
mse_loss = nn.MSELoss()



from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch

def create_train_dataloader(dfTrain, batch_size=64,time_steps=60):
    # Number of features (excluding the target column "emission")
    num_features = 74

    # Extracting features and target
    features = dfTrain.drop(columns=['emission']).values
    target = dfTrain['emission'].values

    # Creating sequences of 60 timesteps
    sequences = []
    labels = []
    for i in range(len(features) - time_steps):
        sequences.append(features[i:i+time_steps])
        labels.append(target[i+time_steps])

    # Converting to PyTorch tensors
    sequences_tensor = torch.tensor(sequences, dtype=torch.float32).view(-1, time_steps, num_features)
    labels_tensor = torch.tensor(labels, dtype=torch.float32).view(-1, 1)

    # Creating a TensorDataset and DataLoader
    train_dataset = TensorDataset(sequences_tensor, labels_tensor)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    return train_dataloader

def create_test_dataloader(dfTest, batch_size=64):
    # Number of features
    num_features = 74

    # Extracting features
    features = dfTest.values

    # Creating sequences of 60 timesteps
    sequences = [features[i:i+60] for i in range(len(features) - 60)]

    # Converting to PyTorch tensors
    sequences_tensor = torch.tensor(sequences, dtype=torch.float32).view(-1, 60, num_features)

    # Creating a TensorDataset and DataLoader
    test_dataset = TensorDataset(sequences_tensor)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return test_dataloader



In [ ]:
from google.colab import files

def create_and_download_submission(predictions, filename='submission.csv'):
    # Read the CSV file that contains the ID of each test sample
    dfid = pd.read_csv(folder_path + 'testR.csv')

    # Create a dataframe for the submission
    submission = pd.DataFrame({
        "ID_LAT_LON_YEAR_WEEK": dfid["ID_LAT_LON_YEAR_WEEK"],
        "emission": predictions
    })

    # Save the submission dataframe to a CSV file
    submission.to_csv(filename, index=False)

    # Download the CSV file
    files.download(filename)

In [ ]:
class LSTM_Arch(nn.Module):
    def __init__(self, input_size,
                 hidden_sizes,  # List of hidden sizes
                 dropouts,      # List of dropouts
                 attention_size):
        super(LSTM_Arch, self).__init__()
        num_layers = len(hidden_sizes)  # Determine the number of layers from the hidden_sizes list
        self.lstms = nn.ModuleList([nn.LSTM(input_size if i == 0 else hidden_sizes[i - 1],
                                            hidden_sizes[i],
                                            num_layers=1,
                                            batch_first=True,
                                            dropout=0)  # Set dropout to 0 since num_layers is 1
                                    for i in range(num_layers)])
        self.attentions = nn.ModuleList([nn.Linear(hidden_sizes[i], attention_size) for i in range(num_layers)])
        self.attention_combines = nn.ModuleList([nn.Linear(attention_size, 1) for i in range(num_layers)])
        self.fc = nn.Linear(hidden_sizes[-1], 1)



    def apply_attention(self, out, attention_layer, attention_combine_layer):
        attention_weights = F.softmax(attention_combine_layer(F.tanh(attention_layer(out))), dim=1)
        out = torch.bmm(attention_weights.transpose(1, 2), out)
        return out.squeeze(1)

    def forward(self, x):
        for i in range(len(self.lstms)):3
            out, _ = self.lstms[i](x if i == 0 else out.unsqueeze(1))
            out = self.apply_attention(out, self.attentions[i], self.attention_combines[i])
        out = self.fc(out)
        return out


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

def train(model, optimizer, train_dataloader, device, mse_loss, n=10):
    separator = '~' * 180
    print(' ' * 67 + 'TRAINING HAS STARTED')
    print()
    print(separator)

    model.train()
    total_loss = 0
    total_preds = []
    total_labels = []
    chars_on_line = 0
    for step, batch in enumerate(train_dataloader):
        features, labels = batch[0].to(device), batch[1].to(device)
        model.zero_grad()
        preds = model(features)
        loss = mse_loss(preds, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        total_preds.append(preds.detach().cpu().numpy())
        total_labels.append(labels.detach().cpu().numpy())

        if (step + 1) % n == 0:
            step_chars = len(str(step + 1)) + 1
            chars_on_line += step_chars
            print(step + 1, end='|')
            if chars_on_line + step_chars > len(separator):
                print()
                print(separator)
                chars_on_line = 0
    print()
    print(separator)
    print()

    avg_loss = total_loss / len(train_dataloader)
    total_preds = np.concatenate(total_preds, axis=0)
    total_labels = np.concatenate(total_labels, axis=0)

    rmse = np.sqrt(mean_squared_error(total_labels, total_preds))
    mae = mean_absolute_error(total_labels, total_preds)
    mape = mean_absolute_percentage_error(total_labels, total_preds)
    r2 = r2_score(total_labels, total_preds)


    return avg_loss, total_preds, rmse, mae, mape, r2


def evaluate(model, val_dataloader, device, mse_loss, n=10):
    separator = '~' * 180
    print()
    print(' ' * 67 + 'EVALUATION HAS STARTED')
    print()
    print(separator)

    model.eval()
    total_loss = 0
    total_preds = []
    total_labels = []
    chars_on_line = 0
    for step, batch in enumerate(val_dataloader):
        features, labels = batch[0].to(device), batch[1].to(device)
        with torch.no_grad():
            preds = model(features)
            loss = mse_loss(preds, labels)
            total_loss += loss.item()
            total_preds.append(preds.detach().cpu().numpy())
            total_labels.append(labels.detach().cpu().numpy())

        if (step + 1) % n == 0:
            step_chars = len(str(step + 1)) + 1
            chars_on_line += step_chars
            print(step + 1, end='|')
            if chars_on_line + step_chars > len(separator):
                print()
                print(separator)
                chars_on_line = 0
    print()
    print(separator)
    print()

    avg_loss = total_loss / len(val_dataloader)
    total_preds = np.concatenate(total_preds, axis=0)
    total_labels = np.concatenate(total_labels, axis=0)

    rmse = np.sqrt(mean_squared_error(total_labels, total_preds))
    mae = mean_absolute_error(total_labels, total_preds)
    mape = mean_absolute_percentage_error(total_labels, total_preds)
    r2 = r2_score(total_labels, total_preds)



    return avg_loss, total_preds, rmse, mae, mape, r2



In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import AdamW
import torch.nn.functional as F


from math import sqrt

import copy
class LSTM_Arch(nn.Module):
    def __init__(self, input_size,
                 hidden_sizes,  # List of hidden sizes
                 dropouts,      # List of dropouts
                 attention_size):
        super(LSTM_Arch, self).__init__()
        num_layers = len(hidden_sizes)  # Determine the number of layers from the hidden_sizes list
        self.lstms = nn.ModuleList([nn.LSTM(input_size if i == 0 else hidden_sizes[i - 1],
                                            hidden_sizes[i],
                                            num_layers=1,
                                            batch_first=True,
                                            dropout=0)  # Set dropout to 0 since num_layers is 1
                                    for i in range(num_layers)])
        self.attentions = nn.ModuleList([nn.Linear(hidden_sizes[i], attention_size) for i in range(num_layers)])
        self.attention_combines = nn.ModuleList([nn.Linear(attention_size, 1) for i in range(num_layers)])
        self.fc = nn.Linear(hidden_sizes[-1], 1)



    def apply_attention(self, out, attention_layer, attention_combine_layer):
        attention_weights = F.softmax(attention_combine_layer(F.tanh(attention_layer(out))), dim=1)
        out = torch.bmm(attention_weights.transpose(1, 2), out)
        return out.squeeze(1)

    def forward(self, x):
        for i in range(len(self.lstms)):
            out, _ = self.lstms[i](x if i == 0 else out.unsqueeze(1))
            out = self.apply_attention(out, self.attentions[i], self.attention_combines[i])
        out = self.fc(out)
        return out

best_state_dict = None
best_params = None
best_global_metric=float('inf')

def objective(trial, dfTrain, device,layers=10):
    global best_state_dict, best_params,best_global_metric
    separator = '~' * 180
    model_params = {
        "lr": [3e-8, 1e-2],
        "weight_decay": [1e-6, 1e-2],
        "dropout": [0.1, 0.5],
        "beta1": [0.8, 0.99],
        "beta2": [0.9, 0.9999],
        "factor": [0.05, 0.5],
        "scheduler_patience": [1, 2],
        "threshold": [1e-5, 1e-3],
        "cooldown": [0, 1],
        "min_lr": [0, 1e-3],

        "time_steps":[50,5000],
        "batch_size": [128, 256],
        "epochs": [10, 15],
        "attention_size": [128, 512],
        'test_size':[0.15,0.45],
        'random_state':[42,1024],
}

    for i in range(1, layers+1):
        model_params[f"hidden_size{i}"] = [256, 512]
        model_params[f"dropout{i}"] = [0.1, 0.5]


    params = {}
    for param, bounds in model_params.items():
        if isinstance(bounds[0], int):
            params[param] = trial.suggest_int(param, min(bounds), max(bounds))
        elif isinstance(bounds[0], float):
            params[param] = trial.suggest_float(param, min(bounds), max(bounds))

    for param, value in params.items():
        print(f'{param} = {value}')

    train_data, val_data = train_test_split(dfTrain, test_size=params["test_size"], random_state=params["random_state"])

    # Create data loaders for training and validation data
    train_dataloader = create_train_dataloader(train_data, batch_size=params["batch_size"],time_steps=params["time_steps"])
    val_dataloader = create_train_dataloader(val_data, batch_size=params["batch_size"],time_steps=params["time_steps"])  # Using the same function for validation data

    # Define input_size for the RNN model
    input_size = 74



    hidden_sizes = [params[f"hidden_size{i}"] for i in range(1, layers+1)]
    dropouts = [params[f"dropout{i}"] for i in range(1, layers+1)]

    # Instantiate the model using the new LSTM_Arch class
    model = LSTM_Arch(
        input_size=input_size,
        hidden_sizes=hidden_sizes,
        dropouts=dropouts,
        attention_size=params["attention_size"]
    )

    model = model.to(device)

    optimizer = AdamW(model.parameters(),
                      lr=params["lr"],
                      betas=(params["beta1"], params["beta2"]),
                      #eps=params["eps_optimizer"],
                      weight_decay=params["weight_decay"])

    scheduler = ReduceLROnPlateau(optimizer,
                                  mode='min',
                                  factor=params["factor"],
                                  patience=params["scheduler_patience"],
                                  verbose=False,
                                  threshold=params["threshold"],
                                  threshold_mode='rel',
                                  cooldown=params["cooldown"],
                                  min_lr=params["min_lr"],
                                  #eps=params["scheduler_eps"]
                                  )


    mse_loss = nn.MSELoss()

    best_train_loss = float('inf')
    best_local_metric = float('inf')
    separator = '=' * 180
    metric=0


    for epoch in range(params["epochs"]):
        print('-----------------------')
        print(f'    |Epoch {epoch + 1} / {params["epochs"]}|')
        print('-----------------------')

        train_loss, train_preds,train_rmse, train_mae, train_mape, train_r2 = train(model, optimizer, train_dataloader, device, mse_loss)
        valid_loss, val_preds,val_rmse, val_mae, val_mape, val_r2 = evaluate(model, val_dataloader, device, mse_loss)


        print("Training Metrics:")
        print(f"Average Loss: {train_loss}")
        print(f"RMSE: {train_rmse}")
        print(f"MAE: {train_mae}")
        print(f"MAPE: {train_mape}")
        print(f"R² Score: {train_r2}")
        print(separator)
        print()


        print("Training Metrics:")
        print(f"Average Loss: {val_loss}")
        print(f"RMSE: {val_rmse}")
        print(f"MAE: {val_mae}")
        print(f"MAPE: {val_mape}")
        print(f"R² Score: {val_r2}")
        print(separator)
        print()



        metric = (valid_loss - train_loss)**2 * (valid_loss**2 + train_loss**2) * train_loss


        if metric < best_local_metric:
            best_local_metric = metric
            if best_local_metric < best_global_metric :
                best_local_metric = metric
                best_state_dict = copy.deepcopy(model.state_dict())
                best_params = params  # Save the parameters used to create the best model
                print('new best found')
            print('-----------------------')
        best_train_loss = min(train_loss, best_train_loss)
        print(f'Training Loss: {train_loss:.3f}')
        print(f'Validation Loss: {valid_loss:.3f}')

    print('*' * 180)
    print('                                                                                    TRIAL=GUD')
    print('*' * 180)
    print()




    torch.save(model.state_dict(), f'saved_weights_trial_{trial.number}.pt')

    print(('-----------------------'))

    return best_local_metric




def tune_model_with_optuna(dfTrain, device, layers=10):
    global best_state_dict, best_params,best_global_metric
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, dfTrain, device, layers=layers), n_trials=10)

    trial = study.best_trial

    if trial.state == optuna.trial.TrialState.COMPLETE:
        print(f'Best trial found with value: {trial.value:.3f}')
        print('Best parameters:')
        for key, value in trial.params.items():
            print(f'    {key}: {value}')
    else:
        print('No completed trials.')

    input_size = dfTrain.drop(columns=['emission']).shape[1]

    hidden_sizes = [best_params[f"hidden_size{i}"] for i in range(1, layers + 1)]
    dropouts = [best_params[f"dropout{i}"] for i in range(1, layers + 1)]

    best_model = LSTM_Arch(
        input_size=input_size,
        hidden_sizes=hidden_sizes,
        dropouts=dropouts,
        attention_size=best_params["attention_size"]
    )

    best_model.load_state_dict(best_state_dict)
    best_model = best_model.to(device)


    return best_model



In [ ]:

folder_path = '/content/drive/MyDrive/datasets/'
dfTrain = pd.read_csv(folder_path + 'trainR.csv')
dfTest = pd.read_csv(folder_path + 'testR.csv')
dfTrain.drop(['ID_LAT_LON_YEAR_WEEK'],axis=1,inplace=True)

dfTest.drop(['ID_LAT_LON_YEAR_WEEK'],axis=1,inplace=True)
dfTrain=fill_nans_with_previous(dfTrain)
dfTest=fill_nans_with_previous(dfTest)

In [ ]:
best_state_dict = None
best_params = None
best_global_metric=float('inf')

tune_model_with_optuna(dfTrain, device)

[I 2023-08-14 18:43:27,684] A new study created in memory with name: no-name-1173d2f9-a8f4-4d65-af37-d8cb38f9fa0a


#using just models(nornn)

In [ ]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Gradient Boosting Regressor
from sklearn.ensemble import GradientBoostingRegressor




# AdaBoost Regressor
from sklearn.ensemble import AdaBoostRegressor



# XGBoost Regressor
from xgboost import XGBRegressor



# Linear Regression
from sklearn.linear_model import LinearRegression

# Lasso Regression
# Lasso Regression
lasso = Lasso()
lasso_params = {
    'alpha': [0.01, 100],
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'max_iter': [500, 10000],
    'tol': [0.00001, 0.01],
    'warm_start': [True, False],
    'positive': [True, False],
    'selection': ['cyclic', 'random']
}

# Ridge Regression
ridge = Ridge()
ridge_params = {
    'alpha': [0.01, 100],
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'max_iter': [500, 2000],
    'tol': [0.00001, 0.01],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
}

# Random Forest Regressor
random_forest = RandomForestRegressor()
rf_params = {
    'n_estimators': [25, 100],
    'max_depth': [3, 20],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 10],
    'max_features': ['sqrt'],
    #'bootstrap': [True, False],
    #'oob_score': [False, True],
    #'warm_start': [False, True]
}

# Gradient Boosting Regressor
gradient_boosting = GradientBoostingRegressor()
gbr_params = {
    'loss': ['ls', 'lad', 'huber', 'quantile'],
    'learning_rate': [0.001, 0.5],
    'n_estimators': [25, 100],
    'subsample': [0.1, 1.0],
    'warm_start': [False, True],
    'validation_fraction': [0.05, 0.5],
    'tol': [1e-5, 1e-3]
}

# AdaBoost Regressor
adaboost = AdaBoostRegressor()
ada_params = {
    'n_estimators': [25, 100],
    'learning_rate': [0.001, 0.5],
    'loss': ['linear', 'square', 'exponential']
}
xgboost = XGBRegressor()
xgboost_params ={
    'n_estimators': (50, 200),
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.5),
    'gamma': (0, 0.5),
    'reg_lambda': (0.1, 1.0),
    'reg_alpha': (0.1, 1.0),
    'subsample': (0.5, 1.0)
    }



# Linear Regression
linear_regression = LinearRegression()
lr_params = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'positive': [False, True]
}


# Linear Regression
linear_regression = LinearRegression()
lr_params = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'positive': [False, True]
}

lgbm=LGBMRegressor()

lgbm_params = {
    'n_estimators': (200, 300),
    'learning_rate': (0.01, 0.5),
    'boosting_type': ['gbdt', 'dart', 'rf'],
    'num_leaves': (51, 200),
    'max_depth': (3, 10),
    'subsample_for_bin': (20000, 300000),
    'min_split_gain': (0.0, 1.0),
    'min_child_weight': (1e-3, 1e-1),
    'min_child_samples': (5, 50),
    'subsample': (0.5, 1.0),
    'subsample_freq': (0, 10),
    'colsample_bytree': (0.5, 1.0),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0)
}
xgboost = XGBRegressor()
xgboost_params ={
    'n_estimators': (50, 200),
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.5),
    'gamma': (0, 0.5),
    'reg_lambda': (0.1, 1.0),
    'reg_alpha': (0.1, 1.0),
    'subsample': (0.5, 1.0)
    }



In [ ]:
lgbm=LGBMRegressor()

lgbm_params = {
    'n_estimators': (216, 300),
    'learning_rate': (0.1086, 0.4925),
    'boosting_type': ['dart', 'gbdt'],
    'num_leaves': (101, 200),
    'max_depth': (8, 12),
    'subsample_for_bin': (21762, 57851),
    'min_split_gain': (0.00299, 0.0997),
    'min_child_weight': (0.00533, 0.0195),
    'min_child_samples': (10, 20),
    'subsample': (0.8077, 0.9976),
    'subsample_freq': (5, 10),
    'colsample_bytree': (0.5112, 0.9971),
    'reg_alpha': (0.00042, 0.9982),
    'reg_lambda': (0.0527, 0.9851)
}



In [ ]:
import optuna
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
lgbm=LGBMRegressor()

lgbm_params = {
    'n_estimators': (216, 300),
    'learning_rate': (0.1086, 0.4925),
    'boosting_type': ['dart', 'gbdt'],
    'num_leaves': (101, 200),
    'max_depth': (8, 12),
    'subsample_for_bin': (21762, 57851),
    'min_split_gain': (0.00299, 0.0997),
    'min_child_weight': (0.00533, 0.0195),
    'min_child_samples': (10, 20),
    'subsample': (0.8077, 0.9976),
    'subsample_freq': (5, 10),
    'colsample_bytree': (0.5112, 0.9971),
    'reg_alpha': (0.00042, 0.9982),
    'reg_lambda': (0.0527, 0.9851)
}

class ModelSelectionPipeline:
    def __init__(self, model, model_params, n_jobs=-1, n_trials=15):
        self.model = model
        self.model_params = model_params
        self.n_jobs = n_jobs
        self.best_model = None
        self.best_predictions = None
        self.n_trials=n_trials

    def fit_and_predict(self, X_train, y_train, X_test):



        print(f"Evaluating model: {self.model.__class__.__name__}")
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: self.objective(trial, X_train, y_train),  n_trials=self.n_trials)
        print(f"Best trial for model {self.model.__class__.__name__}: score {study.best_value}, params {study.best_params}")


        self.best_model = self.model.set_params(**study.best_params)
        self.best_model.fit(X_train, y_train)
        self.best_predictions = self.best_model.predict(X_test)

    def objective(self, trial, X, y):
        params = self.model.get_params(deep=True)  # Start with the model's default parameters
        for param, bounds in self.model_params.items():
            if isinstance(bounds[0], int):
                params[param] = trial.suggest_int(param, min(bounds), max(bounds))
            elif isinstance(bounds[0], float):
                params[param] = trial.suggest_float(param, min(bounds), max(bounds))
            else:
                params[param] = trial.suggest_categorical(param, bounds)
        self.model.set_params(**params)

        # Use MAE as the scoring metric
        scores = cross_val_score(self.model, X, y, cv=10, scoring='neg_mean_absolute_error', n_jobs=self.n_jobs)

        return -scores.mean()  # Minimize the negative mean absolute error



def best_preds_comb(models, param_grids, X_train_combined, X_test_combined, y, n_trials=15):
    # Split the full training data into train, meta_train and validation sets
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_train_combined, y, test_size=0.30, random_state=42)
    X_meta_train, X_val, y_meta_train, y_val = train_test_split(
        X_temp, y_temp, test_size=(6/30), random_state=42)

    train_predictions = []
    meta_train_predictions = []
    val_predictions = []
    test_predictions = []
    model_selections = []

    for model, param_grid in zip(models, param_grids):
        pipeline = ModelSelectionPipeline(model, param_grid, n_trials=n_trials)
        pipeline.fit_and_predict(X_train, y_train, X_meta_train)

        train_predictions.append(pipeline.best_model.predict(X_train))
        meta_train_predictions.append(pipeline.best_model.predict(X_meta_train))
        val_predictions.append(pipeline.best_model.predict(X_val))
        test_predictions.append(pipeline.best_model.predict(X_test_combined))
        model_selections.append(pipeline.best_model)


    meta_param_grid  = {
    'n_estimators': (150, 300),
    'learning_rate': (0.1, 0.5), # Narrowing down based on best value
    'boosting_type': ['gbdt', 'dart'], # Removing 'rf' as 'dart' performed best
    'num_leaves': (100, 200), # Narrowing down based on best value
    'max_depth': (8, 12), # Narrowing down based on best value
    'subsample_for_bin': (20000, 60000), # Narrowing down based on best value
    'min_split_gain': (0.0, 0.1), # Narrowing down based on best value
    'min_child_weight': (0.005, 0.02), # Narrowing down based on best value
    'min_child_samples': (10, 20), # Narrowing down based on best value
    'subsample': (0.8, 1.0), # Narrowing down based on best value
    'subsample_freq': (5, 10), # Keeping the same, as varied results
    'colsample_bytree': (0.5, 1.0),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0)
    }

    meta_model = LGBMRegressor()

    meta_train_predictions_np = np.column_stack(meta_train_predictions)
    test_predictions_np = np.column_stack(test_predictions)

    pipeline = ModelSelectionPipeline(meta_model, meta_param_grid, n_trials=15)
    pipeline.fit_and_predict(meta_train_predictions_np, y_meta_train, test_predictions_np)

    # Calculate the mean squared error of the meta model on the validation data
    val_predictions_np = np.column_stack(val_predictions)
    meta_model_mse = mean_squared_error(y_val, pipeline.best_model.predict(val_predictions_np))
    print(f"Meta model mean squared error: {meta_model_mse}")

    return pipeline.best_predictions, model_selections



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

class ModelSelectionPipeline:
    def __init__(self, model, model_params, n_jobs=-1, n_trials=15):
        self.model = model
        self.model_params = model_params
        self.n_jobs = n_jobs
        self.best_model = None
        self.best_predictions = None
        self.n_trials=n_trials

    def fit_and_predict(self, X_train, y_train, X_test):



        print(f"Evaluating model: {self.model.__class__.__name__}")
        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: self.objective(trial, X_train, y_train),  n_trials=self.n_trials)
        print(f"Best trial for model {self.model.__class__.__name__}: score {study.best_value}, params {study.best_params}")


        self.best_model = self.model.set_params(**study.best_params)
        self.best_model.fit(X_train, y_train)
        self.best_predictions = self.best_model.predict(X_test)

    def objective(self, trial, X, y):
        params = self.model.get_params(deep=True)  # Start with the model's default parameters
        for param, bounds in self.model_params.items():
            if isinstance(bounds[0], int):
                params[param] = trial.suggest_int(param, min(bounds), max(bounds))
            elif isinstance(bounds[0], float):
                params[param] = trial.suggest_float(param, min(bounds), max(bounds))
            else:
                params[param] = trial.suggest_categorical(param, bounds)
        self.model.set_params(**params)

        # Use MAE as the scoring metric
        scores = cross_val_score(self.model, X, y, cv=10, scoring='neg_mean_absolute_error', n_jobs=self.n_jobs)

        return -scores.mean()  # Minimize the negative mean absolute error

def best_preds_one(models, param_grids, X_train_combined, X_test_combined, y, n_trials=10):
    # Split the combined training data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_combined, y, test_size=0.2, random_state=42)

    test_predictions = []
    last_predictions = None

    for model, param_grid in zip(models, param_grids):
        pipeline = ModelSelectionPipeline(model, param_grid, n_trials=n_trials)
        pipeline.fit_and_predict(X_train, y_train, X_test_combined)

        pipeline_mse = mean_squared_error(y_val, pipeline.best_model.predict(X_val))
        print(f"Model mean squared error: {pipeline_mse}")



        test_predictions.append(pipeline.best_model.predict(X_test_combined))
        print('------------------------------------------------------------------------')

        last_predictions = pipeline.best_model.predict(X_test_combined)

    return test_predictions, last_predictions


In [ ]:
models = [
    #linear_regression,
    #lasso, # changed from lasso_regression
    #ridge, # changed from ridge_regression
    #random_forest, # changed from random_forest_regressor
    #gradient_boosting, # changed from gradient_boosting_regressor
    #adaboost, # changed from adaboost_regressor
    #xgboost, # changed from xgboost_regressor
    lgbm
]

param_grids = [
    #lr_params,
    #lasso_params,
    #ridge_params,
    #rf_params,
    #gbr_params,
    #ada_params,
    #xgboost_params, # changed from xgb_params
    lgbm_params
]

# Reading data
folder_path = '/content/drive/MyDrive/datasets/'
dfTrain = pd.read_csv(folder_path + 'trainR.csv')
dfTest = pd.read_csv(folder_path + 'testR.csv')

dfTrain.drop(['ID_LAT_LON_YEAR_WEEK'],axis=1,inplace=True)

dfTest.drop(['ID_LAT_LON_YEAR_WEEK'],axis=1,inplace=True)
dfTrain =fill_nans_with_previous(dfTrain)

dfTest =fill_nans_with_previous(dfTest)



In [ ]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 75 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   latitude                                                  79023 non-null  float64
 1   longitude                                                 79023 non-null  float64
 2   year                                                      79023 non-null  int64  
 3   week_no                                                   79023 non-null  int64  
 4   SulphurDioxide_SO2_column_number_density                  79023 non-null  float64
 5   SulphurDioxide_SO2_column_number_density_amf              79023 non-null  float64
 6   SulphurDioxide_SO2_slant_column_number_density            79023 non-null  float64
 7   SulphurDioxide_cloud_fraction                             79023 non-null  float64
 8   SulphurDioxide_s

In [ ]:

#lag days feature
def previous(df, column_num=3, num_days=1):
    # Copy the dataframe
    df_copy = df.copy()

    for i in range(1, num_days + 1):
        # Shift the specified column 'i' days into the past
        shifted_series = df_copy.iloc[:, column_num].shift(i)
        # Create a new column with the shifted values
        df_copy.insert(i - 1, f'Price_{i}_ago', shifted_series)

    # Remove the first 'num_days' rows from the copied dataframe
    df_copy = df_copy[num_days:]

    # Fill NaN values with the most recent valid value
    df_copy.fillna(method='ffill', inplace=True)

    return df_copy


In [ ]:
predictions, model_selected = best_preds_comb(models, param_grids,
                                              dfTrain.drop(['emission'], axis=1),
                                              dfTest,
                                              dfTrain['emission'])



In [ ]:
predictions,lp = best_preds_one(models, param_grids,

                                              dfTrain.drop(['emission'], axis=1),
                                              dfTest,
                                              dfTrain['emission'])


[I 2023-08-10 18:25:05,675] A new study created in memory with name: no-name-73a95ec3-50bf-4755-a4a4-809befcabc87


Evaluating model: LGBMRegressor


[I 2023-08-10 18:27:49,999] Trial 0 finished with value: 9.262423923325667 and parameters: {'n_estimators': 259, 'learning_rate': 0.2791486107843115, 'boosting_type': 'dart', 'num_leaves': 174, 'max_depth': 8, 'subsample_for_bin': 24391, 'min_split_gain': 0.044494203709832245, 'min_child_weight': 0.01843602150273977, 'min_child_samples': 12, 'subsample': 0.9071602230753402, 'subsample_freq': 7, 'colsample_bytree': 0.9257723875537283, 'reg_alpha': 0.409611674807905, 'reg_lambda': 0.38049000266969213}. Best is trial 0 with value: 9.262423923325667.
[I 2023-08-10 18:30:38,026] Trial 1 finished with value: 9.152497063813183 and parameters: {'n_estimators': 297, 'learning_rate': 0.4329765813769375, 'boosting_type': 'dart', 'num_leaves': 119, 'max_depth': 8, 'subsample_for_bin': 37984, 'min_split_gain': 0.09935134102521778, 'min_child_weight': 0.007647331362348591, 'min_child_samples': 19, 'subsample': 0.8183922569615208, 'subsample_freq': 7, 'colsample_bytree': 0.8464920537693446, 'reg_alph

Best trial for model LGBMRegressor: score 7.366383768936852, params {'n_estimators': 271, 'learning_rate': 0.4618486912974318, 'boosting_type': 'dart', 'num_leaves': 190, 'max_depth': 12, 'subsample_for_bin': 36786, 'min_split_gain': 0.050698282988378404, 'min_child_weight': 0.015661242771447002, 'min_child_samples': 17, 'subsample': 0.8578090482304607, 'subsample_freq': 8, 'colsample_bytree': 0.864798188564072, 'reg_alpha': 0.03273212593245682, 'reg_lambda': 0.579316577117079}
Model mean squared error: 335.6011782100024
------------------------------------------------------------------------


In [ ]:
len(predictions[0])

24353

In [ ]:
from google.colab import files

def create_and_download_submission(predictions, filename='submission.csv'):
    # Read the CSV file that contains the ID of each test sample
    dfid = pd.read_csv(folder_path + 'testR.csv')

    # Create a dataframe for the submission
    submission = pd.DataFrame({
        "ID_LAT_LON_YEAR_WEEK": dfid["ID_LAT_LON_YEAR_WEEK"],
        "emission": predictions
    })

    # Save the submission dataframe to a CSV file
    submission.to_csv(filename, index=False)

    # Download the CSV file
    files.download(filename)

In [ ]:
create_and_download_submission(predictions[0], 'submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>